In [ ]:
import os
import logging
import json
from time import sleep
from IPython.core.display import display, HTML, Image

# Little helpers

def printAsHtmlTable(objList, keys):
    htmlRows = []
    htmlRows.append("<table>")
    htmlRows.append("<tbody>")
    
    htmlRows.append("<tr>")
    for k in keys:
        htmlRows.append("<th>")
        htmlRows.append(k)
        htmlRows.append("</th>")
    htmlRows.append("</tr>")
        
    for obj in objList:
        htmlRows.append("<tr>")
        for k in keys:
            htmlRows.append("<td>")
            htmlRows.append(obj.get(k, "<i>[NULL]</i>"))
            htmlRows.append("</td>")
        htmlRows.append("</tr>")
    htmlRows.append("</tbody>")
    htmlRows.append("</table>")
    
    display(HTML('\n'.join(htmlRows)))
    
def printDevices(devices):
    printAsHtmlTable(devices, ["Id", "Model", "Name", "Status"])

# Getting credentials and codes
These values are taken from config.json, if it exists.
```
$ cat config.json
{
    "avs_username": "test_username",
    "avs_password": "test_password",
    "pairing_code": "9999-1111",
    "rtmp_url": "rtmp://youtube.com/live/test"
}

$ jupyter nbconvert --to html --execute device_api.ipynb --ExecutePreprocessor.timeout=-1
```

Alternatively, you can hardcode those values in the code below.

In [ ]:
kAVSUsername = "test_username"
kAVSPassword = "test_password"
kPairingCode = "9999-1111"
kRTMPUrl = "rtmp://youtube.com/live/test"

configFilename = "config.json"
if os.path.isfile(configFilename):
    with open(configFilename) as f:
        cfg = json.load(f)

        kAVSUsername = cfg["avs_username"]
        kAVSPassword = cfg["avs_password"]
        kPairingCode = cfg["pairing_code"]
        kRTMPUrl = cfg["rtmp_url"]

# Importing AV Studio API

In [ ]:
from avstudio import AVStudioAPI

# Establishing connection to AV Studio

In [ ]:
api = AVStudioAPI("go.avstudio.com")

In [ ]:
logger = logging.getLogger("avstudio")
logging.basicConfig()
logger.setLevel(logging.ERROR) # use logging.DEBUG to see raw HTTP

In [ ]:
api.login(kAVSUsername, kAVSPassword)
allDevices = api.Devices.get_all()
    
printDevices(allDevices)

# Pairing device to AV Studio:

In [ ]:
deviceId = api.Devices.add(kPairingCode, "Webcaster on my desk")["ID"]

In [ ]:
printDevices(api.Devices.get_all())

# Getting current image from the device

In [ ]:
sleep(2) # Bit of delay after pairing

tmpImagePath = "/tmp/image_%s.jpg" % deviceId
api.Devices.get_state_image(deviceId, tmpImagePath)
Image(tmpImagePath)

# Setting stream parameters

In [ ]:
api.Devices.run_command(deviceId, "setparam:bitrate=%d" % 1024)  # 1 Mbit/s
api.Devices.run_command(deviceId, 'setparam:resolution=%s' % "640x360")

devTelemetry = api.Devices.get(deviceId)["Telemetry"]
if "settings" in devTelemetry:
    printAsHtmlTable(devTelemetry["settings"].values(), ["id", "title", "value"])

# Streaming to RTMP
To start RTMP stream, use `rtmp.start` command:

In [ ]:
_ = api.Devices.run_command(deviceId, "rtmp.start:%s" % kRTMPUrl)

## Getting RMTP stream status

Device reports its status every 30 seconds and after receiving a command. To get the most recent device status, send `nop` command: device will do nothing, but it will force status submission.

In [ ]:
sleep(2) # Let it start

api.Devices.run_command(deviceId, "nop")

rtmp_status = api.Devices.get(deviceId)["Telemetry"]["state"]["rtmp"]
print(json.dumps(rtmp_status, indent=4))

## Stopping RTMP streaming

In [ ]:
_ = api.Devices.run_command(deviceId, "rtmp.stop")

# Unpairing the device:

In [ ]:
api.Devices.delete(deviceId)
printDevices(api.Devices.get_all())